Python 对重载运算符施加了一些限制，做好了灵活性、可用性和安全性方面的平衡：
- 不能重载内置类型的运算符
- 不能新建运算符，只能重载现有的
- 某些运算符不能重载：is、and、or、not

一元运算符|对应的特殊方法
-|-
-|\_\_neg__
+|\_\_pos__
~|\_\_invert__
abs()|\_\_abs__

支持一元运算符很简单，只需实现相应的特殊方法，这些特殊方法只有一个参数 self 。要遵守运算符的一个基本规则：始终返回一个新对象，即不能修改 self ，要创建并返回合适类型的新实例。

In [1]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    # 一元运算符：abs()
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    # 一元运算符：-
    def __neg__(self):
        return Vector(-x for x in self)
    
    # 一元运算符：+
    def __pos__(self):
        return Vector(self)
    
    # 加法运算符
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)  # 能处理任何可迭代对象
        return Vector(a + b for a, b in pairs)

    
    # 加法运算符的反射/反向
    def __radd__(self, other):
        return self + other

In [2]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [3]:
v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [4]:
v3 = Vector([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0])

实现一元运算符和中缀运算符的特殊方法一定不能修改操作数。使用这些运算符的表达式期待的结果是新对象。只有增量赋值表达式可能会修改第一个操作数（self）

为了支持涉及不同类型的运算，Python 为中缀运算符特殊方法提供了特殊的分派机制。

\_\_radd__ 是 \_\_add__ 的反射（reflected）或反向（reversed）版本。这是一种后备机制，如果左操作数没有实现 \_\_add__ 方法，或者实现了，但是返回 NotImplemented 表明它不知道如何处理右操作数，那么 Python 会调用 \_\_radd__ 方法。

\_\_radd__ 通常就这么简单：直接调用适当的运算符，在这里就是委托 \_\_add__ 。任何交换的运算符都能这么做。

In [5]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [6]:
# __add__ 方法的操作数应该是可迭代的数值对象
v1 + 1

TypeError: 'int' object is not iterable

如果由于类型不兼容而导致运算符特殊方法无法返回有效的结果，那么应该返回 NotImplemented ，而不是抛出 TypeError 。

为了遵守鸭子精神，我们不能测试 other 操作数的类型，或者它的元素的类型。我们要捕获异常，然后返回 NotImplemented 。

In [7]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # 超球面坐标
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    # 一元运算符：abs()
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    # 一元运算符：-
    def __neg__(self):
        return Vector(-x for x in self)
    
    # 一元运算符：+
    def __pos__(self):
        return Vector(self)
    
    # 加法运算符
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)  # 能处理任何可迭代对象
            return Vector(a + b for a, b in pairs)
        except TypeError:  # 捕获异常
            return NotImplemented
    
    # 加法运算符的反射/反向
    def __radd__(self, other):
        return self + other

In [8]:
v1 = Vector([3, 4, 5])
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

标量积（scalar product）/元素级乘法（elementwise multiplication）：各个分量都乘以 x 。

点积（dot product）：如果把一个向量看作 1xN 矩阵，把另一个向量看作 Nx1 矩阵，那么就是矩阵乘法。
- @ 记号可以用作中缀点积运算符

白鹅类型的实际运用 —— 显式检查抽象类型
- decimal.Decimal 没有把自己注册为 numbers.Real 的虚拟子类

In [9]:

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # 超球面坐标
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
    
    # 支持抽象基类的真实子类、虚拟子类
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    # 委托给 __mul__
    def __rmul__(self, scalar):
        return self * scalar
    
    # 点积 @
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
    
    # 委托给 __matmul__
    def __rmatmul__(self, other):
        return self @ other

In [10]:
v1 = Vector([1.0, 2.0, 3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [11]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [12]:
from fractions import Fraction

v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

运算符|正向方法|反向方法|就地方法|说明
:-|:-|:-|:-|:-
+|\_\_add__|\_\_radd__|\_\_iadd__|加法或拼接
-|\_\_sub__|\_\_rsub__|\_\_isub__|减法
\*|\_\_mul__|\_\_rmul__|\_\_imul__|乘法或重复复制
/|\_\_truediv__|\_\_rtruediv__|\_\_itruediv__|除法
//|\_\_floordiv__|\_\_rfloordiv__|\_\_ifloordiv__|整除
%|\_\_mod__|\_\_rmod__|\_\_imod__|取模
divmod()|\_\_divmod__|\_\_rdivmod__|\_\_idivmod__|返回由整除的商和模数组成的元组
\*\*, pow()|\_\_pow__|\_\_rpow__|\_\_ipow__|取幂
@|\_\_matmul__|\_\_rmatmul__|\_\_imatmul__|矩阵乘法
&|\_\_and__|\_\_rand__|\_\_iand__|位与
\||\_\_or__|\_\_ror__|\_\_ior__|位或
^|\_\_xor__|\_\_rxor__|\_\_ixor__|位异或
<<|\_\_lshift__|\_\_rlshift__|\_\_ilshift__|按位左移
>>|\_\_rshift__|\_\_rrshift__|\_\_irshift__|按位右移

- pow 的第三个参数 modulo 是可选的：pow(a, b, modulo)，直接调用特殊方法时也支持这个参数，例如 a.\_\_pow__(b, modulo)
- @ Python 3.5 新引入

In [13]:
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz == 38.0

True

In [14]:
va @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

比较运算符
- 正向和反向调用使用的是同一系列方法
- 对 == 和 != 来说，如果反向调用失败，Python 会比较对象的 ID ，而不抛出 TypeError

中缀运算符|正向方法调用|反向方法调用|后备机制
:-|:-|:-|:-
a == b|a.\_\_eq__(b)|b.\_\_eq__(a)|返回 id(a) == id(b)
a != b|a.\_\_ne__(b)|b.\_\_ne__(a)|返回 not(a==b)
a > b|a.\_\_gt__(b)|b.\_\_lt__(a)|抛出 TypeError
a < b|a.\_\_lt__(b)|b.\_\_gt__(a)|抛出 TypeError
a >= b|a.\_\_ge__(b)|b.\_\_le__(a)|抛出 TypeError
a <= b|a.\_\_le__(b)|b.\_\_ge__(a)|抛出 TypeError

对于 \_\_ne__ ，现在 Python 3 返回结果是对 \_\_eq__ 结果的取反。对于排序比较运算符，Python 3 抛出 TypeError ，并把错误消息设置为 `'unorderable types: int() < tuple()'`

In [15]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [16]:
# 不应该是元组
t3 = (1, 2, 3)
va == t3

True

Python 之禅：如果存在多种可能，不要猜测。

\_\_eq__ 中添加了类型检查

In [17]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # 超球面坐标
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar
    
    
    def __eq__(self, other):
        if isinstance(other, Vector):  # 如果是 Vector 或其子类的实例
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented  # 否则，返回 NotImplemented

In [18]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [19]:
t3 = (1, 2, 3)
va == t3

False

对 == 来说，如果反向调用返回 NotImplemented ，Python 会比较对象的 ID 。而 != 运算符，由从 object 继承的 \_\_ne__ 方法的后备行为实现：定义了 \_\_eq__ 方法，而且它不返回 NotImplemented，\_\_ne__ 会对 \_\_eq__ 返回的结果取反。

In [20]:
va != vb

False

In [21]:
va != (1, 2, 3)

True

增量运算符不会修改不可变目标，而是新建实例，然后重新绑定。

如果一个类没有实现上面表格列出的就地运算符，增量赋值运算符只是语法糖：a += b 的作用与 a = a + b 完全一样。对不可变类型来说，一定不能实现就地特殊方法。

In [22]:
v1 = Vector([1, 2, 3])
v1_alias = v1  # 复制
id(v1)

1605983051440

In [23]:
v1 += Vector([4, 5, 6])
v1

Vector([5.0, 7.0, 9.0])

In [24]:
id(v1)

1605983164400

In [25]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [26]:
v1 *= 11
v1

Vector([55.0, 77.0, 99.0])

In [27]:
id(v1)

1605983164544

In [28]:
import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>


    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))

In [29]:
import random


class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <7>
        self.pick()

In [30]:
import itertools

# 扩展 BingoCage
class AddableBingoCage(BingoCage):

    def __add__(self, other):
        if isinstance(other, Tombola):  # 第二个操作数只能是 Tombola 实例
            return AddableBingoCage(self.inspect() + other.inspect())  # 获取元素
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)  # 使用 other 创建迭代器
            except TypeError:  # 抛出异常
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self  # 增值赋值特殊方法必须返回 self

In [31]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)  # 使用 5 个元素创建实例
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [32]:
globe.pick() in vowels  # 取出元素

True

In [33]:
len(globe.inspect())  # 元素数量减少

4

In [34]:
globe2 = AddableBingoCage('XYZ')  # 使用 3 个元素创建实例
globe3 = globe + globe2
len(globe3.inspect())  # 7 个元素

7

In [35]:
# 无法与列表相加
# __add__ 方法返回 NotImplemented
void = globe + [10, 20]

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [36]:
globe_orig = globe  # 复制
len(globe.inspect())

4

In [37]:
# 从同属一类的其他实例那里接受元素
globe += globe2
len(globe.inspect())

7

In [38]:
# 右操作数可以是任何迭代对象
globe += ['M', 'N']
len(globe.inspect())

9

In [39]:
# 不变
globe is globe_orig

True

In [40]:
# 不能与非可迭代对象相加
globe += 1

TypeError: right operand in += must be 'AddableBingoCage' or an iterable

与 + 相比，+= 运算符对第二个操作数更宽容。因为就地修改左操作数，所以结果的类型是确定的。

一般来说，如果中缀运算符的正向方法（如 \_\_mul__）只处理与 self 属于同一类型的操作数，那就无需实现对应的反向方法（如 \_\_rmul__）

禁止重载内置类型的运算符，而且限于重载现有的运算符，例外：is、and、or、not

鸭子类型更灵活，但是显式检查更能预知结果。

Python 还会特别处理 == 和 != 的后备机制：从不抛出错误，因为 Python 会比较对象的 ID 。

对序列类型来说，+ 通常要求两个操作数属于同一类型，而 += 的右操作数往往可以是任何可迭代对象。

泛函数，由 Python 3 的 @singledispatch 装饰器支持。

生成器表达式在最后时刻才会计算，而不是在源码中定义它的位置计算。